# Post processing

In [7]:
import pandas as pd
import numpy as np
from model_creation import buildNN

## Data Loading


In [2]:
# Reading rating file
dataset = pd.read_csv("games_data.csv")
#Convert user and game ids from int to categories
dataset['user_id'] = dataset['userID'].astype('category').cat.codes.values
dataset['game_id'] = dataset['gameID'].astype('category').cat.codes.values

## Building Model

In [3]:
n_users, n_games = len(dataset.user_id.unique()), len(dataset.game_id.unique())
n_latent_factors = 500
model=buildNN(n_users, n_games, n_latent_factors)
model.load_weights('weights.h5')

In [4]:
def get_game_ids(userID, rating):
    """
    This method takes a user id and rating and returns
    data frame contains only the games ids that the 
    user has watched and rated the given rating
    """
    real_user_id = dataset.loc[(dataset['userID'] == userID)]['userID'].iloc[0]
    df = dataset.loc[(dataset['userID'] == real_user_id)]
    return list(df[dataset['rating']== rating]['gameID'])

In [5]:
def recommend_games(userID, top_n):
    played_games = dataset[dataset["user_id"] == userID]["game_id"]
    played_games_set = set(played_games.tolist())
    #preparing recommendations
    predictedRatingsDF = pd.DataFrame(columns=['game_id','rating'])
    for game_id in list(set(dataset.game_id)):
        if game_id in played_games_set:
            continue
        predictedRating = model.predict([np.array([userID]),np.array([game_id])])[0][0]
        real_game_id = dataset.loc[(dataset['game_id'] == game_id)]['gameID'].iloc[0]
        predictedRatingsDF.loc[predictedRatingsDF.shape[0]]=pd.Series({"game_id":real_game_id,
                                                                       "rating":predictedRating})
    df = predictedRatingsDF.sort_values(by=['rating'], ascending=False).reset_index(drop=True)[:top_n]
    games_names = []
    for i in range(len(df)):
        game_id = df.iloc[i, 0]
        games_names.append(dataset[dataset["gameID"] == game_id].iloc[0, 1])
    df["gameName"] = games_names
    return df

In [23]:
dataset[dataset.user_id==5088]

,userID,gameName,rating,gameID,user_id,game_id
0,151603712,The Elder Scrolls V Skyrim,5.000000,100000.0,5088,0
1,151603712,Fallout 4,4.919359,100001.0,5088,1
2,151603712,Spore,3.897582,100002.0,5088,2
3,151603712,Fallout New Vegas,3.777047,100003.0,5088,3
4,151603712,Left 4 Dead 2,3.599187,100004.0,5088,4
5,151603712,HuniePop,3.572559,100005.0,5088,5
6,151603712,Path of Exile,3.544647,100006.0,5088,6
7,151603712,Poly Bridge,3.500082,100007.0,5088,7
8,151603712,Left 4 Dead,3.024685,100008.0,5088,8
9,151603712,Team Fortress 2,2.929544,100009.0,5088,9


In [22]:
recommend_games(userID=5088, top_n=10)

,game_id,rating,gameName
0,100054.0,3.501667,Rocket League
1,100671.0,3.467769,The Witcher 3 Wild Hunt
2,100046.0,3.297688,Grand Theft Auto V
3,100052.0,3.220290,Far Cry 3
4,100048.0,3.219394,METAL GEAR SOLID V THE PHANTOM PAIN
5,100874.0,3.215882,Assassin's Creed IV Black Flag
6,100944.0,3.198008,NBA 2K15
7,100473.0,3.157445,The Sims(TM) 3
8,100055.0,3.154450,Far Cry 4
9,100767.0,3.137713,DARK SOULS II
